In [19]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

from sklearn.metrics import f1_score
# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/stacking_pkl_file/*.pkl')):
    if 'fm2.pkl' in feat or 'ligbm' in  feat or '2leve' in  feat :
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
    
# train_x = np.nan_to_num(np.hstack(train_x))
# test_x = np.nan_to_num(np.hstack(test_x))
# print(train_x.shape)
    
# load y

# list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
# train_y = train[list_classes].values.astype('int')
# print(train_x.shape)

file path ../feature/stacking_pkl_file/Capsule.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_all_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/dpcnn2000.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/pseudo_rnn2.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn2000.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn7879.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_all_shuffle_all.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn

In [20]:
for feat in sorted(glob.glob('../feature/other_features/features-vinson/*.csv')):
    if 'fm2.pkl' in feat or 'w2v' in feat or 'lda' in feat or 'lsi' in feat or 'xgb' in feat:
        continue
    print('file path',feat)
    file=pd.read_csv(feat)
    a = file.iloc[:102277,:]
    b = file.iloc[102277:,:]
    a = a.values
    b = b.values
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/other_features/features-vinson/bnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob_0.778620.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/mnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/nn_ensemble_0.775597.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob_0.778881.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/word_fasttext_0.760225.csv
(102277, 19) (102277, 19)
(102277, 437)


In [21]:
import pandas as pd
import numpy as np
train = pd.read_csv("../input/new_data/train_set.csv")
y=(train["class"]-1).astype(int)
del train
import gc
gc.collect()

458

In [22]:

from keras.utils import np_utils
train_y=np_utils.to_categorical(y,num_classes=19)

In [23]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat, average='macro'), True

In [24]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred, val_pred = np.zeros((102277,20)), np.zeros((102277,20))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_f1_l,all_val_f1_l = 0,0
    
    for i in range(19):
        val_loss_l,train_loss_l = 0,0
        val_f1_l,train_f1_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x
            # share params
            params = {
                    'application': 'binary',
                    'num_leaves': 15,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': 4,
                    #'scale_pos_weight':0.9,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    'bagging_fraction': bagging_fraction,
                    'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,
                    'num_threads':4

                    }
            if met == 'auc':
                s_round = 100
            else:
                s_round = 50
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
#                       feval=lgb_f1_score,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=100,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            train_pred = model.predict(curr_x)
            val_pred[test_index, i] = model.predict(hold_out_x)
            curr_train_loss = log_loss(curr_y[:,i],train_pred)
            curr_val_loss = log_loss(hold_out_y[:,i],val_pred[test_index, i])

            curr_train_f1 = self_f1_score(curr_y[:,i],train_pred)
            curr_val_f1 = self_f1_score(hold_out_y[:,i],val_pred[test_index, i])

            print('ls',curr_train_loss,curr_val_loss,'f1',curr_train_f1,curr_val_f1)
            val_loss_l += curr_val_loss
            train_loss_l += curr_train_loss
            val_f1_l += curr_val_f1
            train_f1_l += curr_train_f1
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_f1_l = train_f1_l/k
        val_f1_l = val_f1_l/k
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class f1 train',train_f1_l,'f1 val',val_f1_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/19
        all_val_loss_l += val_loss_l/19
        all_train_f1_l += train_f1_l/19
        all_val_f1_l += val_f1_l/19
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all f1 avg',all_train_f1_l,all_val_f1_l)
    print('=======================================================')
    return val_pred, test_pred

print('done')

done


In [25]:

lgb_train, lgb_res = simple_ens('lgb',5,233,0.05,0.6)
##all loss avg 0.04941417146346469 0.05321999547256359
# all f1 avg 0.8965591064486441 0.8915637951017253
#######################7982<lb<7985

#all loss avg 0.04911572582626101 0.052974216091397554
# all f1 avg 0.8970629823676124 0.8922944850147942

# all loss avg 0.0414203008656468 0.052978859434622785
# all f1 avg 0.9103949564755508 0.8924077581728779

# all loss avg 0.04539159633943475 0.05255053671449584
# all f1 avg 0.9029697229659733 0.8934845973558836

#all loss avg 0.04494700768702617 0.052254864071516055
# all f1 avg 0.9040207644649525 0.8939220820764944

# all loss avg 0.04353723194221981 0.051953447288746665
# all f1 avg 0.9064007590628727 0.8947753048612783
# 7940810057586943

# all loss avg 0.041791878911084705 0.0502085007027876
# all f1 avg 0.9094424575179055 0.8970108064794169
#0.7956058839820572
#0.8979044788487316

/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:186: UserWarning: Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.
  warnings.warn("Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.")


0 fold: ls 0.08827563701288117 0.09553616387429105 f1 0.8043222755634124 0.7994799611204153
1 fold: ls 0.08657027530838257 0.09877635784900357 f1 0.8067430705685795 0.7896281263712155
2 fold: ls 0.08881709534546858 0.09729638105085076 f1 0.8049214364250823 0.8037576017451533
3 fold: ls 0.08017932511270662 0.09831054426410638 f1 0.8204449311485509 0.7942332855894818
4 fold: ls 0.07941667528772789 0.0975385637618827 f1 0.8196513936465029 0.7943812764818012
this class avg train 0.08465180161343337 avg val 0.09749160216002689
this class f1 train 0.8112166214704256 f1 val 0.7962960502616134
0 fold: ls 0.024078599889330036 0.028995300020772382 f1 0.9138724808527812 0.9035573194311969
1 fold: ls 0.0218544243459654 0.03236329900675408 f1 0.9186785020445116 0.8936851796734968
2 fold: ls 0.023874336909500783 0.029480033848739182 f1 0.9154258963494506 0.9042401162149432
3 fold: ls 0.022012829601353045 0.03255242649172084 f1 0.9191003363656742 0.8932971667559146
4 fold: ls 0.02359262807802929 0.03

2 fold: ls 0.04936849461141498 0.0594056258789206 f1 0.9188920908357143 0.8999406558669256
3 fold: ls 0.049601417257087804 0.060595329909789546 f1 0.918950635457121 0.9043372428641226
4 fold: ls 0.04819868363151262 0.06091689271508531 f1 0.9185476624366806 0.9083890166922003
this class avg train 0.04870699962802454 avg val 0.0590846736383551
this class f1 train 0.9192765987623558 f1 val 0.9063863854144962
0 fold: ls 0.025842413181468883 0.031953902163249795 f1 0.9652093361930522 0.9604460777317545
1 fold: ls 0.02514372216349045 0.0314766734781418 f1 0.9669257643847755 0.9599539282346058
2 fold: ls 0.025990628438587637 0.03659816193842366 f1 0.9658653113943834 0.9532230104503185
3 fold: ls 0.02650726287590244 0.03511565768984758 f1 0.9652708198303492 0.9620627565069513
4 fold: ls 0.027157055141466154 0.03455296424411928 f1 0.9644849470857366 0.9584981762462647
this class avg train 0.02612821636018311 avg val 0.033939471902756424
this class f1 train 0.9655512357776594 f1 val 0.9588367898

In [26]:
with open('../feature/2leve_lgb_stacking8.pkl','wb') as fout:
    pickle.dump([lgb_train,lgb_res],fout)

In [27]:
test_dir = '../input/new_data/test_set.csv'
test = pd.read_csv(test_dir)
test_id = test[["id"]].copy()

In [28]:
preds=np.argmax(lgb_res,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('../output/lgbstacknnx3.csv',index=None)

(102277, 1)
(102277, 1)


In [29]:
preds

array([ 5,  4, 13, ...,  6, 14, 13])

In [30]:
name = ["class_prob_%s"%i for i in range(1,21)]
df_lgb = pd.DataFrame(lgb_res, columns=name)
df_lgb = df_lgb.drop('class_prob_20', axis=1)

In [31]:

df_lgb.to_csv('../pro/stacking_lgbx3.csv', index=False)

In [32]:
df_lgb

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19
0,0.002243,0.000325,0.000705,0.003167,0.989950,0.001370,0.000524,0.000191,0.000360,0.000538,0.001770,0.000976,0.000614,0.000613,0.000310,0.000070,0.000260,0.000324,0.000731
1,0.002102,0.000295,0.000464,0.990493,0.000269,0.000269,0.000282,0.000248,0.000218,0.000324,0.000426,0.001487,0.000343,0.000186,0.000174,0.000054,0.000215,0.000347,0.000537
2,0.048086,0.003451,0.090661,0.002261,0.016101,0.002078,0.010597,0.057101,0.005158,0.005243,0.015374,0.043277,0.674879,0.005587,0.008171,0.000305,0.007258,0.005865,0.024633
3,0.000416,0.000195,0.000306,0.993631,0.000303,0.000304,0.000210,0.000234,0.000195,0.000369,0.000226,0.000319,0.000299,0.000133,0.000169,0.000053,0.000230,0.000154,0.000491
4,0.007421,0.000248,0.001962,0.003248,0.957149,0.000700,0.007263,0.000243,0.000522,0.004221,0.000996,0.000768,0.001649,0.012623,0.001140,0.000132,0.000462,0.000778,0.002011
5,0.042360,0.004619,0.004972,0.000883,0.905513,0.004797,0.000374,0.012448,0.002398,0.014645,0.009018,0.006934,0.001465,0.000992,0.000831,0.000456,0.000345,0.004316,0.003913
6,0.022126,0.001004,0.004279,0.004812,0.000478,0.000994,0.000616,0.002421,0.000458,0.073293,0.000976,0.007860,0.001686,0.003009,0.900470,0.000592,0.001519,0.007326,0.033831
7,0.027287,0.150816,0.002845,0.001222,0.003192,0.009782,0.072876,0.010019,0.003041,0.018198,0.004696,0.009411,0.030359,0.004425,0.008250,0.000220,0.084286,0.032652,0.567641
8,0.000297,0.000154,0.993834,0.000148,0.000278,0.000350,0.000192,0.000134,0.000251,0.000239,0.000597,0.000672,0.000638,0.000140,0.000178,0.000051,0.000194,0.000205,0.000327
9,0.116432,0.002801,0.001755,0.002846,0.002998,0.003100,0.018618,0.150082,0.014089,0.153835,0.008002,0.515543,0.023741,0.001925,0.000579,0.000924,0.001923,0.018156,0.078827


In [8]:
a[0]

array([0.63021084, 0.00073022, 0.12938255, 0.08233588, 0.38772137,
       0.35054736, 0.06010154, 0.0162588 , 0.35044424, 0.36899349,
       0.33343102, 0.00718196, 0.00099249, 0.13392986, 0.03079003,
       0.40782321, 0.38772949, 0.11120969, 0.07217844, 0.        ])

In [17]:
b[0]

array([0.10306741, 0.21446534, 0.09234781, 0.1665037 , 0.77700784,
       0.20140753, 0.09591953, 0.09388762, 0.21980566, 0.2528162 ,
       0.18957717, 0.15716459, 0.16162135, 0.08285841, 0.18494502,
       0.15308998, 0.16854106, 0.1316501 , 0.07221129])

In [13]:
a = np.delete(a, -1, axis=1)

In [12]:
a, b= pickle.load(open('../feature/stacking_pkl_file/2leve_lgb_stacking3.pkl','rb'))

In [12]:
b = pickle.load(open('../feature/stacking_pkl_file/rnn_model_bagging2.pkl','rb'))

In [5]:
a.shape

(102277, 20)

In [18]:
with open('../feature/2leve_lgb_stacking3.pkl','wb') as fout:
    pickle.dump([a,b],fout)

In [26]:
c.shape

(102277, 19)

In [29]:
c[0]

array([6.01716689e-04, 6.16839170e-05, 7.57186936e-05, 1.06231030e-03,
       9.96983826e-01, 3.12459830e-04, 6.37228004e-05, 1.02987576e-06,
       2.93245284e-05, 1.16468422e-04, 1.56775292e-04, 3.49557318e-04,
       1.02049326e-05, 3.78012592e-05, 1.93889423e-06, 1.43233101e-05,
       1.13495912e-06, 1.05727275e-04, 1.42806357e-05], dtype=float32)

In [ ]:
a=[1,2]

In [11]:
lgb_res[1]

array([7.51418984e-04, 8.59006145e-05, 5.72015080e-05, 9.98149313e-01,
       2.16004989e-05, 2.21425307e-05, 1.92076409e-05, 2.88491225e-05,
       4.80360194e-05, 6.77559877e-05, 1.46532588e-04, 8.52455299e-05,
       1.89921750e-05, 8.53312692e-06, 1.33990754e-05, 7.04179440e-06,
       1.30948036e-05, 1.99016595e-05, 7.09543232e-05, 0.00000000e+00])